# Zurich Instruments LabOne Python API Example

## Run a frequency sweep with a SHFQA

Run a frequency sweep with a SHFQA.

Requirements:
* LabOne Version >= 21.08
* Instruments:
    1 x SHFQA or SHFQC Instrument
* Signal output 0 connected to signal input 0 with a BNC cable

-----

In [ ]:
import zhinst.core
from zhinst.utils.shf_sweeper import (
    ShfSweeper,
    AvgConfig,
    RfConfig,
    SweepConfig,
)

### 1. Connect the device

Set up the connection with the device. Firstly create an API session with a Data Server. Then the Data Server connects to the device.

In [ ]:
device_id = "dev2345" # Device serial number available on its rear panel.
interface = "1GbE" # For Ethernet connection
#interface = "USB" # For USB connection

server_host = "localhost"
server_port = 8004
api_level = 6 # Maximum API level supported for all instruments.

# Create an API session to the Data Server.
daq = zhinst.core.ziDAQServer(server_host, server_port, api_level)

# Establish a connection between Data Server and Device.
daq.connectDevice(device_id, interface)

### 2. Create a Sweeper and configure it

Instantiate the sweeper.

In [ ]:
sweeper = ShfSweeper(daq, device_id)

Configure the sweeper.

In [ ]:
# Frequency range settings for the sweep
sweep_config = SweepConfig(
    start_freq=-200e6,
    stop_freq=300e6,
    num_points=51,
    mapping="linear",
    oscillator_gain=0.8,
)

# Averaging settings for the sweep
avg_config = AvgConfig(integration_time=100e-6, num_averages=2, mode="sequential")

# RF input and output settings for the sweep
rf_config = RfConfig(channel=0, input_range=0, output_range=0, center_freq=4e9)


sweeper.configure(sweep_config, avg_config, rf_config)


### 3. Run the measurement and plot the data

Turn on the input and output for the channel.

In [ ]:
daq.set(
    [
        (f"/{device_id}/qachannels/{rf_config.channel}/input/on", 1),
        (f"/{device_id}/qachannels/{rf_config.channel}/output/on", 1),
    ]
)

Start a sweep.

In [ ]:
sweeper.run()

Get the result after the sweep.

In [ ]:
result = sweeper.get_result()
num_points_result = len(result["vector"])
print(f"Measured at {num_points_result} frequency points.")

Plot the the result over the frequency.

In [ ]:
sweeper.plot()